In [29]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

### PD Set Options

In [77]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [30]:
dsf1 = tabula.read_pdf('Topcondoimage 07-04-2022.pdf', pages='all',lattice=True)
dsf2 = tabula.read_pdf('Topcondoimage 07-11-2022.pdf', pages='all',lattice=True)
dsf3 = tabula.read_pdf('Topcondoimage 07-18-2022.pdf', pages='all',lattice=True)
dsf4 = tabula.read_pdf('Topcondoimage 07-25-2022.pdf', pages='all',lattice=True)
dsf5 = tabula.read_pdf('Topcondoimage 08-01-2022.pdf', pages='all',lattice=True)
dsf6 = tabula.read_pdf('Topcondoimage 08-08-2022.pdf', pages='all',lattice=True)

In [31]:
df1 = dsf1[0]
df2 = dsf2[0]
df3 = dsf3[0]
df4 = dsf4[0]
df5 = dsf5[0]
df6 = dsf6[0]

In [32]:
df1.columns = df1.iloc[0]
df2.columns = df2.iloc[0]
df3.columns = df3.iloc[0]
df4.columns = df4.iloc[0]
df5.columns = df5.iloc[0]
df6.columns = df6.iloc[0]

In [33]:
df1 = df1.drop(df1.index[0])
df2 = df2.drop(df2.index[0])
df3 = df3.drop(df3.index[0])
df4 = df4.drop(df4.index[0])
df5 = df5.drop(df5.index[0])
df6 = df6.drop(df6.index[0])

In [34]:
df1 = df1.iloc[: , 1:]
df2 = df2.iloc[: , 1:]
df3 = df3.iloc[: , 1:]
df4 = df4.iloc[: , 1:]
df5 = df5.iloc[: , 1:]
df6 = df6.iloc[: , 1:]

In [35]:
df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

In [36]:
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [37]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [38]:
df.loc[len(df.index) + 1] = ['asia_condo','9/9/2020','7/21/2022','673','$7,000,000','square_feet','$1,413','Jordan Lederman','Fortune International Realty','Jorge Rico & Andrea Viana','Compass Florida','geo_address','Asia condo','900 Brickell Key Boulevard, PH3401']

In [39]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

In [40]:
x = df['Close Date'].str.startswith('7')
df = df[x]

In [41]:
len(df)

41

In [43]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

In [44]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [45]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/22809927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)


In [46]:
df['loc']

10    (788 NE 23rd St #1101, Miami, FL 33137, USA, (...
11    (789 Crandon Blvd APT 504, Key Biscayne, FL 33...
12    (21200 NE 38th Ave APT 802, Aventura, FL 33180...
13    (1445 16th St APT 1101, Miami Beach, FL 33139,...
14    (3000 Island Blvd #2503, Aventura, FL 33160, U...
15    (7000 Island Blvd APT 1906, Aventura, FL 33160...
16    (6103 Aqua Ave APT 502, Miami Beach, FL 33141,...
17    (3131 NE 7th Ave #1503, Miami, FL 33137, USA, ...
18    (5225 Collins Ave APT 1221, Miami Beach, FL 33...
19    (9 Island Ave APT 1905, Miami Beach, FL 33139,...
20    (10201 Collins Ave #1801s, Bal Harbour, FL 331...
21    (360 Ocean Dr #302s, Key Biscayne, FL 33149, U...
22    (10155 Collins Ave, Bal Harbour, FL 33154, USA...
23    (2901 S Bayshore Dr #13b, Miami, FL 33133, USA...
24    (718 Valencia Ave Unit 305, Coral Gables, FL 3...
25    (1395 Brickell Ave #3406, Miami, FL 33131, USA...
26    (200 Biscayne Blvd Way APT 1109, Miami, FL 331...
27    (15811 Collins Ave APT 2103, Sunny Isles B

In [47]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/1957931448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)


Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [48]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point'],
      dtype='object')

In [49]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)



/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/3326340165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/3326340165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/3326340165.py:1: SettingWit

In [50]:
df.index

Int64Index([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
            27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
            44, 45, 46, 47, 48, 49, 61],
           dtype='int64')

In [51]:
### Not sure why this works the way it does. But if I want to simply reset the index because the numbers are askew,
### I need to set both 'drop' and 'inplace' to 'True'
df.reset_index(drop=True, inplace=True)

In [52]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
0,The Elysee\r788 NE 23rd St 1101\rMiami,6/1/2022,7/5/2022,2,"$2,970,000.00",3310,"$1,082.76",Felise Eber,Coldwell Banker Realty,Pedro Velasco,Global Luxury Realty LLC,The Elysee 788 NE 23rd St 1101 Miami,The Elysee,788 NE 23rd St 1101 Miami,"(788 NE 23rd St #1101, Miami, FL 33137, USA, (...","(25.7990651, -80.18554160000001, 0.0)",25.799065,-80.185542,0.0
1,Ocean Club\r789 Crandon Blvd 504\rKey Biscayne,2/22/2022,7/6/2022,83,"$2,800,000.00",2500,"$1,120.00",Liz Hogan,"Compass Florida, Inc",Elena Chacon,BHHS EWM Realty,Ocean Club 789 Crandon Blvd 504 Key Biscayne,Ocean Club,789 Crandon Blvd 504 Key Biscayne,"(789 Crandon Blvd APT 504, Key Biscayne, FL 33...","(25.6868145, -80.15900479999999, 0.0)",25.686815,-80.159005,0.0
2,Atlantic 1 at the Point\r21200 NE 38th Ave 802...,3/20/2022,7/5/2022,19,"$1,850,000.00",2733,$676.91,Cassio Galiza PA,Podium Realty Group Inc,Lucas Pucheta,BM2 Realty,Atlantic 1 at the Point 21200 NE 38th Ave 802 ...,Atlantic,1 at the Point 21200 NE 38th Ave 802 Aventura,"(21200 NE 38th Ave APT 802, Aventura, FL 33180...","(25.9730645, -80.1250513, 0.0)",25.973064,-80.125051,0.0
3,Capri South Beach Condo\r1445 16th St 1101\rMi...,2/28/2022,7/5/2022,66,"$1,800,000.00",1128,"$1,595.74",Jeri Jenkins,Coldwell Banker Realty,Pamela Diaz-De-Leon,Brown Harris Stevens,Capri South Beach Condo 1445 16th St 1101 Miam...,Capri South Beach Condo,1445 16th St 1101 Miami Beach,"(1445 16th St APT 1101, Miami Beach, FL 33139,...","(25.7891852, -80.14467069999999, 0.0)",25.789185,-80.144671,0.0
4,Williams Island\r3000 Island Blvd 2503\rAventura,3/28/2022,7/5/2022,23,"$1,525,000.00",2605,$585.41,Nicole Shein,Williams Island Realty LLC,Joel Matus,Williams Island Realty LLC,Williams Island 3000 Island Blvd 2503 Aventura,Williams Island,3000 Island Blvd 2503 Aventura,"(3000 Island Blvd #2503, Aventura, FL 33160, U...","(25.9406051, -80.1344671, 0.0)",25.940605,-80.134467,0.0
5,Williams Island\r7000 Island Blvd 1906\rAventura,5/19/2022,7/7/2022,5,"$1,500,000.00",2390,$627.62,Joel Matus,Williams Island Realty LLC,Nancy Goldfarb,Douglas Elliman,Williams Island 7000 Island Blvd 1906 Aventura,Williams Island,7000 Island Blvd 1906 Aventura,"(7000 Island Blvd APT 1906, Aventura, FL 33160...","(25.9443867, -80.1356782, 0.0)",25.944387,-80.135678,0.0
6,Aqua at Allison Island\r6103 Aqua Ave 502\rMia...,5/19/2022,7/5/2022,6,"$1,285,000.00",1755,$732.19,Luisa Pizzorni,BHHS EWM Realty,Luisa Pizzorni,BHHS EWM Realty,Aqua at Allison Island 6103 Aqua Ave 502 Miami...,Aqua at Allison Island,6103 Aqua Ave 502 Miami Beach,"(6103 Aqua Ave APT 502, Miami Beach, FL 33141,...","(25.8432662, -80.1228728, 0.0)",25.843266,-80.122873,0.0
7,One Paraiso\r3131 NE 7th Ave 1503\rMiami,1/12/2022,7/5/2022,117,"$1,275,000.00",1526,$835.52,Ricardo Wolf,"Wolf Real Estate Group, PLLC",Jonathan Kanarek,One Sotheby's International,One Paraiso 3131 NE 7th Ave 1503 Miami,One Paraiso,3131 NE 7th Ave 1503 Miami,"(3131 NE 7th Ave #1503, Miami, FL 33137, USA, ...","(25.8070469, -80.1855776, 0.0)",25.807047,-80.185578,0.0
8,The Alexander Condo\r5225 Collins Ave 1221\rMi...,11/29/2021,7/5/2022,189,"$1,250,000.00",1800,$694.44,Evelyn Mackenzie,Aria Luxe Inc,Enzo Rosani,Barnes Int'l Realty LLC,The Alexander Condo 5225 Collins Ave 1221 Miam...,The Alexander Condo,5225 Collins Ave 1221 Miami Beach,"(5225 Collins Ave APT 1221, Miami Beach, FL 33...","(25.8298858, -80.12109, 0.0)",25.829886,-80.121090,0.0
9,N0ine Island Ave Condo\r9 Island Ave 1905\rMia...,3/10/2022,7/5/2022,90,"$1,200,000.00",1440,$833.33,Brigitte Lina,One Sotheby's International,Freddie Baigen,Luxe Properties,N0ine Island Ave Condo 9 Island Ave 1905 Miami...,N,0ine Island Ave Condo 9 Island Ave 1905 Miami ...,"(9 Island Ave APT 1905, Miami Beach, FL 33139,...","(25.7900057, -80.1487379, 0.0)",25.790006,-80.148738

In [53]:
df.at[9,'building_name']=('Nine Island')
df.at[19,'building_name']=('Paramount Bay')
df.at[29,'building_name']=('Oceana IV')
df.at[37,'building_name']=('400 Sunny Isles')
df.at[39,'building_name']=('Turnberry Ocean')

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [54]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'July 2022 Top Miami-Dade Condo Sales ')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['Sq. Ft.'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Address: {bold_start}{info[-1]}{bold_end}{new_line}\
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Square Feet: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

m

In [55]:
m.save('index.html')

In [171]:
### Clean DF for graphic ###

In [56]:
df['Sale Price int'] = df['Sale Price']
df['price_per_sqft_int'] = df['price_per_sqft']

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/2911951518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price']
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/2911951518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = df['price_per_sqft']


In [57]:
df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)

df['price_per_sqft_int'] = df['price_per_sqft_int'].replace('\$','',regex=True)
df['price_per_sqft_int'] = df['price_per_sqft_int'].replace(',','',regex=True)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/310194361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/310194361.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/310194361.py:4: SettingWithCopyWarning: 
A value i

In [58]:
df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/879808360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/879808360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])


In [59]:
df['days_on_market'] = pd.to_numeric(df['days_on_market'])
df['Sq. Ft.'] = pd.to_numeric(df['Sq. Ft.'])

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/1544171665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_on_market'] = pd.to_numeric(df['days_on_market'])


ValueError: Unable to parse string "square_feet" at position 40

In [60]:
df['year'] = 2022

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/1453407984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = 2022


In [61]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude',
       'Sale Price int', 'price_per_sqft_int', 'year'],
      dtype='object')

In [78]:
df.sort_values('Sale Price int',ascending=False)

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int,year
10,Oceana Bal Harbour 10201 Collins Ave 1801S Bal Harbour,2/25/2022,7/11/2022,14,"$12,350,000.00",4185,"$2,951.02",Mark Zilbert,Mark Zilbert,Tracy Galya,Douglas Elliman,Oceana Bal Harbour 10201 Collins Ave 1801S Bal Harbour,Oceana Bal Harbour,10201 Collins Ave 1801S Bal Harbour,"(10201 Collins Ave #1801s, Bal Harbour, FL 33154, USA, (25.8951677, -80.12315))","(25.8951677, -80.12315, 0.0)",25.895168,-80.123150,0.0,12350000.0,2951.02,2021
40,asia_condo,9/9/2020,7/21/2022,673,"$7,000,000",square_feet,"$1,413",Jordan Lederman,Fortune International Realty,Jorge Rico & Andrea Viana,Compass Florida,geo_address,Asia condo,"900 Brickell Key Boulevard, PH3401","(900 Brickell Key Blvd, Miami, FL 33131, USA, (25.7684258, -80.18559189999999))","(25.7684258, -80.18559189999999, 0.0)",25.768426,-80.185592,0.0,7000000.0,1413.00,2021
11,Oceana Key Biscayne 360 Ocean Dr 302S Key Biscayne,1/15/2022,7/15/2022,65,"$6,950,000.00",3358,"$2,069.68",Claudio Iglesias,"Brickell Realty Group, LLC",Cassio Galiza PA,"Podium Realty Group, Inc",Oceana Key Biscayne 360 Ocean Dr 302S Key Biscayne,Oceana Key Biscayne,360 Ocean Dr 302S Key Biscayne,"(360 Ocean Dr #302s, Key Biscayne, FL 33149, USA, (25.6942877, -80.1578218))","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0,6950000.0,2069.68,2021
30,Prive North 5000 Island Estates Dr 1408 Aventura,3/31/2022,7/27/2022,85,"$5,000,000.00",3913,"$1,277.79",Bento Queiroz,"Compass Florida, LLC.",Viviane Wolak,One Sotheby's INT'L Realty,Prive North 5000 Island Estates Dr 1408 Aventura,Prive North,5000 Island Estates Dr 1408 Aventura,"(5000 Island Estates Dr #1408, Aventura, FL 33160, USA, (25.9470842, -80.1306458))","(25.9470842, -80.1306458, 0.0)",25.947084,-80.130646,0.0,5000000.0,1277.79,2021
31,Reach Condo 68 SE 6th St 4111 Miami,3/15/2022,7/29/2022,99,"$3,200,000.00",2526,"$1,266.83",Juan Carlos De Campos,Miami New Realty,Elen Oliveira,One World Properties,Reach Condo 68 SE 6th St 4111 Miami,Reach Condo,68 SE 6th St 4111 Miami,"(68 SE 6th St Apt 4111, Miami, FL 33131, USA, (25.7680813, -80.1926812))","(25.7680813, -80.1926812, 0.0)",25.768081,-80.192681,0.0,3200000.0,1266.83,2021
33,Three Tequesta 848 Brickell Key Dr 4603 Miami,3/1/2022,7/25/2022,89,"$3,000,000.00",3506,$855.68,Berenice Elguezabal,Coldwell Banker Realty,Susana Ojeda,Florida Capital Realty,Three Tequesta 848 Brickell Key Dr 4603 Miami,Three Tequesta,848 Brickell Key Dr 4603 Miami,"(848 Brickell Key Dr APT 4603, Miami, FL 33131, USA, (25.7693285, -80.1837717))","(25.7693285, -80.1837717, 0.0)",25.769329,-80.183772,0.0,3000000.0,855.68,2021
32,Bella Mare 6000 Island Blvd 1501 Aventura,4/5/2022,7/25/2022,26,"$3,000,000.00",4109,$730.10,Brandee Goldstein,"FIP Realty Services, LLC",Ronit Shiro,Beachfront Realty Inc,Bella Mare 6000 Island Blvd 1501 Aventura,Bella Mare,6000 Island Blvd 1501 Aventura,"(6000 Island Blvd #1501, Aventura, FL 33160, USA, (25.9427925, -80.13540019999999))","(25.9427925, -80.13540019999999, 0.0)",25.942792,-80.135400,0.0,3000000.0,730.10,2021
0,The Elysee 788 NE 23rd St 1101 Miami,6/1/2022,7/5/2022,2,"$2,970,000.00",3310,"$1,082.76",Felise Eber,Coldwell Banker Realty,Pedro Velasco,Global Luxury Realty LLC,The Elysee 788 NE 23rd St 1101 Miami,The Elysee,788 NE 23rd St 1101 Miami,"(788 NE 23rd St #1101, Miami, FL 33137, USA, (25.7990651, -80.18554160000001))","(25.7990651, -80.18554160000001, 0.0)",25.799065,-80.185542,0.0,2970000.0,1082.76,2021
1,Ocean Club 789 Crandon Blvd 504 Key Biscayne,2/22/2022,7/6/2022,83,"$2,800,000.00",2500,"$1,120.00",Liz Hogan,"Compass Florida, Inc",Elena Chacon,BHHS EWM Realty,Ocean Club 789 Crandon Blvd 504 Key Biscayne,Ocean Club,789 Crandon Blvd 504 Key Biscayne,"(789 Crandon Blvd APT 504, Key Biscayne, FL 33149, USA, (25.6868145, -80.15900479999999)

In [80]:
#df.sort_values('days_on_market',ascending=False)
df.sort_values('days_on_market',ascending=True)

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int,year
18,Akoya 6365 Collins Ave 3507 Miami Beach,6/5/2022,7/11/2022,1,"$1,400,000.00",1322,"$1,059.00",Nils Jucaud,Charles Rutenberg Realty,Gustavo Duquesne,"Compass Florida, LLC",Akoya 6365 Collins Ave 3507 Miami Beach,Akoya,6365 Collins Ave 3507 Miami Beach,"(6365 Collins Ave APT 3507, Miami Beach, FL 33141, USA, (25.8460388, -80.1196447))","(25.8460388, -80.1196447, 0.0)",25.846039,-80.119645,0.0,1400000.0,1059.00,2021
13,Yacht Harbour Condo 2901 S Bayshore Dr 13B Coconut Grove,6/6/2022,7/11/2022,1,"$2,250,000.00",1587,"$1,417.77",Carole Smith,"Compass Florida, LLC",Maite Alvarez,"Compass Florida, LLC",Yacht Harbour Condo 2901 S Bayshore Dr 13B Coconut Grove,Yacht Harbour Condo,2901 S Bayshore Dr 13B Coconut Grove,"(2901 S Bayshore Dr #13b, Miami, FL 33133, USA, (25.7277525, -80.2394939))","(25.7277525, -80.2394939, 0.0)",25.727753,-80.239494,0.0,2250000.0,1417.77,2021
0,The Elysee 788 NE 23rd St 1101 Miami,6/1/2022,7/5/2022,2,"$2,970,000.00",3310,"$1,082.76",Felise Eber,Coldwell Banker Realty,Pedro Velasco,Global Luxury Realty LLC,The Elysee 788 NE 23rd St 1101 Miami,The Elysee,788 NE 23rd St 1101 Miami,"(788 NE 23rd St #1101, Miami, FL 33137, USA, (25.7990651, -80.18554160000001))","(25.7990651, -80.18554160000001, 0.0)",25.799065,-80.185542,0.0,2970000.0,1082.76,2021
22,Villa Harbour Apts Condo 166 Harbor Dr 7C Key Biscayne,6/16/2022,7/22/2022,3,"$2,100,000.00",1408,"$1,491.48",Shane Graber,Graber Realty Group LLC,Shane Graber,Graber Realty Group LLC,Villa Harbour Apts Condo 166 Harbor Dr 7C Key Biscayne,Villa Harbour Apts Condo,166 Harbor Dr 7C Key Biscayne,"(166 Harbor Dr #7c, Key Biscayne, FL 33149, USA, (25.6997741, -80.16801939999999))","(25.6997741, -80.16801939999999, 0.0)",25.699774,-80.168019,0.0,2100000.0,1491.48,2021
5,Williams Island 7000 Island Blvd 1906 Aventura,5/19/2022,7/7/2022,5,"$1,500,000.00",2390,$627.62,Joel Matus,Williams Island Realty LLC,Nancy Goldfarb,Douglas Elliman,Williams Island 7000 Island Blvd 1906 Aventura,Williams Island,7000 Island Blvd 1906 Aventura,"(7000 Island Blvd APT 1906, Aventura, FL 33160, USA, (25.9443867, -80.1356782))","(25.9443867, -80.1356782, 0.0)",25.944387,-80.135678,0.0,1500000.0,627.62,2021
6,Aqua at Allison Island 6103 Aqua Ave 502 Miami Beach,5/19/2022,7/5/2022,6,"$1,285,000.00",1755,$732.19,Luisa Pizzorni,BHHS EWM Realty,Luisa Pizzorni,BHHS EWM Realty,Aqua at Allison Island 6103 Aqua Ave 502 Miami Beach,Aqua at Allison Island,6103 Aqua Ave 502 Miami Beach,"(6103 Aqua Ave APT 502, Miami Beach, FL 33141, USA, (25.8432662, -80.1228728))","(25.8432662, -80.1228728, 0.0)",25.843266,-80.122873,0.0,1285000.0,732.19,2021
27,Trump Tower III 15811 Collins Ave 3803 Sunny Isles Beach,6/14/2022,7/20/2022,6,"$1,550,000.00",1935,$801.03,Anastasia Philippova,London Foster Realty,David Veit PA,Brown Harris Stevens,Trump Tower III 15811 Collins Ave 3803 Sunny Isles Beach,Trump Tower III,15811 Collins Ave 3803 Sunny Isles Beach,"(15811 Collins Ave APT 3803, Sunny Isles Beach, FL 33160, USA, (25.9232597, -80.1218135))","(25.9232597, -80.1218135, 0.0)",25.923260,-80.121814,0.0,1550000.0,801.03,2021
28,Bel-Air on the Ocean 6315 Collins Ave 1203 Miami Beach,5/27/2022,7/18/2022,7,"$1,295,000.00",1126,"$1,150.09",Glen Janney,Cervera Real Estate Inc.,Michele Redlich,Coldwell Banker Realty,Bel-Air on the Ocean 6315 Collins Ave 1203 Miami Beach,Bel-Air on the Ocean,6315 Collins Ave 1203 Miami Beach,"(6315 Collins Ave #1203, Miami Beach, FL 33141, USA, (25.8449695, -80.1204059))","(25.8449695, -80.1204059, 0.0)",25.844970,-80.120406,0.0,1295000.0,1150.09,2021
24,Murano Grande 400 Alton Rd TH-7M Miami Beach,5/23/2022,7/17/2022,10,"$1,730,000.00",1310,"$1,320.61",Tracy Galya,Douglas Elliman,Kristie Weyhmiller,One Sotheby's INT'L Realty,Murano Grande 400 Alton Rd

### Sale price sum

In [64]:
"${:,.2f}".format(df['Sale Price int'].sum())

'$104,554,750.00'

### Sale price average

In [65]:
"${:,.2f}".format(df['Sale Price int'].mean())

'$2,550,115.85'

### Price per sf average

In [66]:
"${:,.2f}".format(df['price_per_sqft_int'].mean())

'$1,070.45'

### Price per square foot min vs. max

In [67]:
"${:,.2f}".format(df['price_per_sqft_int'].max(axis=0))

'$2,951.02'

In [68]:
"${:,.2f}".format(df['price_per_sqft_int'].min(axis=0))

'$585.41'

### Sale price min and max

In [69]:
"${:,.2f}".format(df['Sale Price int'].max(axis=0))

'$12,350,000.00'

In [70]:
"${:,.2f}".format(df['Sale Price int'].min(axis=0))

'$1,175,000.00'

In [71]:
len(df)

41

In [72]:
df['building_address_city'] = df['building_address_city'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_61988/584204004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['building_address_city'] = df['building_address_city'].map(lambda x: re.sub(r'\r', ' ', x))


,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,...,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int,year
0,The Elysee 788 NE 23rd St 1101 Miami,6/1/2022,7/5/2022,2,"$2,970,000.00",3310,"$1,082.76",Felise Eber,Coldwell Banker Realty,Pedro Velasco,...,The Elysee,788 NE 23rd St 1101 Miami,"(788 NE 23rd St #1101, Miami, FL 33137, USA, (...","(25.7990651, -80.18554160000001, 0.0)",25.799065,-80.185542,0.0,2970000.0,1082.76,2022
1,Ocean Club 789 Crandon Blvd 504 Key Biscayne,2/22/2022,7/6/2022,83,"$2,800,000.00",2500,"$1,120.00",Liz Hogan,"Compass Florida, Inc",Elena Chacon,...,Ocean Club,789 Crandon Blvd 504 Key Biscayne,"(789 Crandon Blvd APT 504, Key Biscayne, FL 33...","(25.6868145, -80.15900479999999, 0.0)",25.686815,-80.159005,0.0,2800000.0,1120.00,2022
2,Atlantic 1 at the Point 21200 NE 38th Ave 802 ...,3/20/2022,7/5/2022,19,"$1,850,000.00",2733,$676.91,Cassio Galiza PA,Podium Realty Group Inc,Lucas Pucheta,...,Atlantic,1 at the Point 21200 NE 38th Ave 802 Aventura,"(21200 NE 38th Ave APT 802, Aventura, FL 33180...","(25.9730645, -80.1250513, 0.0)",25.973064,-80.125051,0.0,1850000.0,676.91,2022
3,Capri South Beach Condo 1445 16th St 1101 Miam...,2/28/2022,7/5/2022,66,"$1,800,000.00",1128,"$1,595.74",Jeri Jenkins,Coldwell Banker Realty,Pamela Diaz-De-Leon,...,Capri South Beach Condo,1445 16th St 1101 Miami Beach,"(1445 16th St APT 1101, Miami Beach, FL 33139,...","(25.7891852, -80.14467069999999, 0.0)",25.789185,-80.144671,0.0,1800000.0,1595.74,2022
4,Williams Island 3000 Island Blvd 2503 Aventura,3/28/2022,7/5/2022,23,"$1,525,000.00",2605,$585.41,Nicole Shein,Williams Island Realty LLC,Joel Matus,...,Williams Island,3000 Island Blvd 2503 Aventura,"(3000 Island Blvd #2503, Aventura, FL 33160, U...","(25.9406051, -80.1344671, 0.0)",25.940605,-80.134467,0.0,1525000.0,585.41,2022
5,Williams Island 7000 Island Blvd 1906 Aventura,5/19/2022,7/7/2022,5,"$1,500,000.00",2390,$627.62,Joel Matus,Williams Island Realty LLC,Nancy Goldfarb,...,Williams Island,7000 Island Blvd 1906 Aventura,"(7000 Island Blvd APT 1906, Aventura, FL 33160...","(25.9443867, -80.1356782, 0.0)",25.944387,-80.135678,0.0,1500000.0,627.62,2022
6,Aqua at Allison Island 6103 Aqua Ave 502 Miami...,5/19/2022,7/5/2022,6,"$1,285,000.00",1755,$732.19,Luisa Pizzorni,BHHS EWM Realty,Luisa Pizzorni,...,Aqua at Allison Island,6103 Aqua Ave 502 Miami Beach,"(6103 Aqua Ave APT 502, Miami Beach, FL 33141,...","(25.8432662, -80.1228728, 0.0)",25.843266,-80.122873,0.0,1285000.0,732.19,2022
7,One Paraiso 3131 NE 7th Ave 1503 Miami,1/12/2022,7/5/2022,117,"$1,275,000.00",1526,$835.52,Ricardo Wolf,"Wolf Real Estate Group, PLLC",Jonathan Kanarek,...,One Paraiso,3131 NE 7th Ave 1503 Miami,"(3131 NE 7th Ave #1503, Miami, FL 33137, USA, ...","(25.8070469, -80.1855776, 0.0)",25.807047,-80.185578,0.0,1275000.0,835.52,2022
8,The Alexander Condo 5225 Collins Ave 1221 Miam...,11/29/2021,7/5/2022,189,"$1,250,000.00",1800,$694.44,Evelyn Mackenzie,Aria Luxe Inc,Enzo Rosani,...,The Alexander Condo,5225 Collins Ave 1221 Miami Beach,"(5225 Collins Ave APT 1221, Miami Beach, FL 33...","(25.8298858, -80.12109, 0.0)",25.829886,-80.121090,0.0,1250000.0,694.44,2022
9,N0ine Island Ave Condo 9 Island Ave 1905 Miami...,3/10/2022,7/5/2022,90,"$1,200,000.00",1440,$833.33,Brigitte Lina,One Sotheby's International,Freddie Baigen,...,Nine Island,0ine Island Ave Condo 9 Island Ave 1905 Miami ...,"(9 Island Ave APT 1905, Miami Beach, FL 33139,...","(25.7900057, -80.1487379, 0.0)",25.790006,-80.148738,0.0,1200000.0,833.33,2022


In [73]:
df['year'] = '2021'

In [74]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,...,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int,year
0,The Elysee 788 NE 23rd St 1101 Miami,6/1/2022,7/5/2022,2,"$2,970,000.00",3310,"$1,082.76",Felise Eber,Coldwell Banker Realty,Pedro Velasco,...,The Elysee,788 NE 23rd St 1101 Miami,"(788 NE 23rd St #1101, Miami, FL 33137, USA, (...","(25.7990651, -80.18554160000001, 0.0)",25.799065,-80.185542,0.0,2970000.0,1082.76,2021
1,Ocean Club 789 Crandon Blvd 504 Key Biscayne,2/22/2022,7/6/2022,83,"$2,800,000.00",2500,"$1,120.00",Liz Hogan,"Compass Florida, Inc",Elena Chacon,...,Ocean Club,789 Crandon Blvd 504 Key Biscayne,"(789 Crandon Blvd APT 504, Key Biscayne, FL 33...","(25.6868145, -80.15900479999999, 0.0)",25.686815,-80.159005,0.0,2800000.0,1120.00,2021
2,Atlantic 1 at the Point 21200 NE 38th Ave 802 ...,3/20/2022,7/5/2022,19,"$1,850,000.00",2733,$676.91,Cassio Galiza PA,Podium Realty Group Inc,Lucas Pucheta,...,Atlantic,1 at the Point 21200 NE 38th Ave 802 Aventura,"(21200 NE 38th Ave APT 802, Aventura, FL 33180...","(25.9730645, -80.1250513, 0.0)",25.973064,-80.125051,0.0,1850000.0,676.91,2021
3,Capri South Beach Condo 1445 16th St 1101 Miam...,2/28/2022,7/5/2022,66,"$1,800,000.00",1128,"$1,595.74",Jeri Jenkins,Coldwell Banker Realty,Pamela Diaz-De-Leon,...,Capri South Beach Condo,1445 16th St 1101 Miami Beach,"(1445 16th St APT 1101, Miami Beach, FL 33139,...","(25.7891852, -80.14467069999999, 0.0)",25.789185,-80.144671,0.0,1800000.0,1595.74,2021
4,Williams Island 3000 Island Blvd 2503 Aventura,3/28/2022,7/5/2022,23,"$1,525,000.00",2605,$585.41,Nicole Shein,Williams Island Realty LLC,Joel Matus,...,Williams Island,3000 Island Blvd 2503 Aventura,"(3000 Island Blvd #2503, Aventura, FL 33160, U...","(25.9406051, -80.1344671, 0.0)",25.940605,-80.134467,0.0,1525000.0,585.41,2021
5,Williams Island 7000 Island Blvd 1906 Aventura,5/19/2022,7/7/2022,5,"$1,500,000.00",2390,$627.62,Joel Matus,Williams Island Realty LLC,Nancy Goldfarb,...,Williams Island,7000 Island Blvd 1906 Aventura,"(7000 Island Blvd APT 1906, Aventura, FL 33160...","(25.9443867, -80.1356782, 0.0)",25.944387,-80.135678,0.0,1500000.0,627.62,2021
6,Aqua at Allison Island 6103 Aqua Ave 502 Miami...,5/19/2022,7/5/2022,6,"$1,285,000.00",1755,$732.19,Luisa Pizzorni,BHHS EWM Realty,Luisa Pizzorni,...,Aqua at Allison Island,6103 Aqua Ave 502 Miami Beach,"(6103 Aqua Ave APT 502, Miami Beach, FL 33141,...","(25.8432662, -80.1228728, 0.0)",25.843266,-80.122873,0.0,1285000.0,732.19,2021
7,One Paraiso 3131 NE 7th Ave 1503 Miami,1/12/2022,7/5/2022,117,"$1,275,000.00",1526,$835.52,Ricardo Wolf,"Wolf Real Estate Group, PLLC",Jonathan Kanarek,...,One Paraiso,3131 NE 7th Ave 1503 Miami,"(3131 NE 7th Ave #1503, Miami, FL 33137, USA, ...","(25.8070469, -80.1855776, 0.0)",25.807047,-80.185578,0.0,1275000.0,835.52,2021
8,The Alexander Condo 5225 Collins Ave 1221 Miam...,11/29/2021,7/5/2022,189,"$1,250,000.00",1800,$694.44,Evelyn Mackenzie,Aria Luxe Inc,Enzo Rosani,...,The Alexander Condo,5225 Collins Ave 1221 Miami Beach,"(5225 Collins Ave APT 1221, Miami Beach, FL 33...","(25.8298858, -80.12109, 0.0)",25.829886,-80.121090,0.0,1250000.0,694.44,2021
9,N0ine Island Ave Condo 9 Island Ave 1905 Miami...,3/10/2022,7/5/2022,90,"$1,200,000.00",1440,$833.33,Brigitte Lina,One Sotheby's International,Freddie Baigen,...,Nine Island,0ine Island Ave Condo 9 Island Ave 1905 Miami ...,"(9 Island Ave APT 1905, Miami Beach, FL 33139,...","(25.7900057, -80.1487379, 0.0)",25.790006,-80.148738,0.0,1200000.0,833.33,2021


In [49]:
df.to_csv('June2022CondoSales.csv', sep='|')

## Map URL Snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [75]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [76]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CondoSalesJuly2022
